In [2]:
from filter_datasets import *
import json
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

## 1. Get MassIVE

In [ ]:
databases_massIVE = getAllDts() #get all datesets available on massIVE
MassIVE_datasets_df = pd.DataFrame(databases_massIVE)
MassIVE_datasets_df.species[MassIVE_datasets_df.species.str.contains('3702')] # Arabidopsis positive control. Search made through taxon ID
MassIVE_datasets_df.head()

In [ ]:
MassIVE_datasets_df.columns

In [ ]:
# massive_plantid_proteomics.tsv is the result from filterMassIVE function on MassIVE databases
MassIVE_passed_df = pd.read_csv('massive_plantid_proteomics.tsv', sep='\t')
MassIVE_passed_df.head()

In [ ]:
#Tried to find zmtable on massIVE datasets. 
for dataset in MassIVE_passed_df["dataset"]:  
    massive_obj = ppx.MassiveProject(
        f"{dataset}")
    massive_remote_files = massive_obj.remote_files(glob = ".mztable")
    if massive_remote_files:
        print(massive_remote_files)
    

## 2. GET PRIDE

In [3]:
# pride_plant.json is the result from filterPride function on PRIDE databases
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total of plants retrieved from pride database: {len(pride_plant)}")

Total of plants retrieved from pride database: 1464


In [4]:
projetc_identifiers = [] # holds project number for all plants species data from pride
accession_number = [] # holds taxon id for all plants species data from pride
for key,value in pride_plant.items():
    projetc_identifiers.append(key)
    accession_number.append(value['accession'])
    

In [5]:
projetc_identifiers[:5]

['PRD000037', 'PRD000044', 'PRD000051', 'PRD000084', 'PRD000096']

In [11]:
accession_number[:5]

['3055', '3702', '4577', '3702', '3702']

In [34]:
# pride changed its FPT link, so the ppx brokes
proj = ppx.find_project(projetc_identifiers[0])
proj_remote_files = proj.remote_files()


error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [53]:
# Try to accesss files through link available on metadata file
with open('pride.log.json',"w+") as pride_log_file:
    for all_projects in projetc_identifiers[:10]:
        proj = ppx.find_project(all_projects)
        print(proj.id)
        print(proj.metadata["_links"]["files"]["href"])
        json.dump(proj.metadata,pride_log_file) # generates a log files in case manual checks in necessary. Takes some time


# remote_files = proj.remote_files()
# print(remote_files)

PRD000037
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000037/files
PRD000044
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000044/files
PRD000051
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000051/files
PRD000084
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000084/files
PRD000096
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000096/files
PRD000114
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000114/files
PRD000116
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000116/files
PRD000119
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000119/files
PRD000126
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000126/files
PRD000133
https://www.ebi.ac.uk/pride/ws/archive/v2/projects/PRD000133/files


In [ ]:
findmztab(projetc_identifiers[0])==None # select m/z tables from a given project.  

In [ ]:
#Search for project with mztab in pride plants database
for i in projetc_identifiers:
    if findmztab(i)!= None: #Selec only those files with m\z tables
        print(i)
        break #stops at fist result 

In [57]:
#Explores the first result
#Everything from here must be done to all findings 
proj = ppx.find_project('PRD000084')

In [48]:
proj.metadata[""]


{'accession': 'PRD000084',
 'title': 'Pollen proteome map of Arabidopsis thaliana',
 'additionalAttributes': [{'@type': 'CvParam',
   'cvLabel': 'PRIDE',
   'accession': 'PRIDE:0000411',
   'name': 'Dataset FTP location',
   'value': 'ftp://ftp.ebi.ac.uk/pride-archive/2012/01/PRD000084'}],
 'projectDescription': 'Not available',
 'sampleProcessingProtocol': 'Not available',
 'dataProcessingProtocol': 'Not available',
 'projectTags': [],
 'keywords': ['Not available'],
 'doi': '',
 'submissionType': 'PRIDE',
 'submissionDate': '2008-10-29',
 'publicationDate': '2012-01-31',
 'submitters': [{'title': 'Dr',
   'affiliation': 'Institute of  Plant Science',
   'email': 'bkatja@ethz.ch',
   'country': '',
   'orcid': '',
   'name': 'Katja Baerenfaller',
   'id': '430136'}],
 'labPIs': [],
 'affiliations': ['Institute of  Plant Science'],
 'instruments': [{'@type': 'CvParam',
   'cvLabel': 'MS',
   'accession': 'MS:1000447',
   'name': 'LTQ'},
  {'@type': 'CvParam',
   'cvLabel': 'MS',
   'ac

In [56]:
pride_plant['PRD000084']

{'name': 'Arabidopsis thaliana (mouse-ear cress)', 'accession': '3702'}

In [55]:
remote_files = proj.remote_files()
print(remote_files)

error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [ ]:
findmztab(i)

In [ ]:
downloaded = proj.download("generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz")
print(downloaded)

In [ ]:
!gzip -df /home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz 

In [ ]:
!ls /home/tiago/.ppx/PRD000084/generated/

In [ ]:
tables = mztab.MzTab("/home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab")
psms = tables.spectrum_match_table
psms.head()

In [ ]:
tables.keys()

In [ ]:
prt = pd.DataFrame(tables['PRT'])

In [ ]:
prt.head()

In [ ]:
prt['description'].str.contains('unknown').sum() #Number of unknown proteins confirmed 

In [ ]:
prt["description"].str.contains("putative").sum() #Number of putative proteins 

In [ ]:
prt["description"].str.contains("hypothetical").sum() #Nuber of hypothetical proteins

In [ ]:
prt['description'].str.contains('lnc').sum() #Number of long non coding RNA?

In [ ]:
prt['description'].str.contains('coding', case=False).sum() #Number of coding RNA?

In [ ]:
prt['description'].str.contains('non', case=False).sum() #Number of coding RNA

In [ ]:
idx = prt['description'].str.contains('unknown') #see the proteins found
idx

In [ ]:
idx[0] = False
prt[idx]

In [ ]:
accession = list(set(accession))

print(len(accession))
accession[:10]

In [ ]:
ncbi = NCBITaxa()


tree = ncbi.get_topology(accession[:3])

print (tree.get_ascii(attributes=["sci_name", "rank"]))

# Not working, make plot of species available, can use other tools
# https://phylot.biobyte.de
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180

for n in tree.traverse():
    if n.is_leaf():
        n.name = n.sci_name

tree.render("pride_plant_tree.pdf", w=500, tree_style=ts)